<a href="https://colab.research.google.com/github/mayank-deeds/nlp-projects/blob/main/mobile_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!nvidia-smi

In [5]:
import tensorflow as tf

import os
import datetime
import tensorflow_datasets as tfds

In [6]:
import pkg_resources


In [7]:
for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
    print(entry_point.dist)

tensorboard 2.7.0
tensorboard-plugin-wit 1.8.0


In [8]:
print(tf.__version__)

2.7.0


In [9]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
dataset, info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', with_info=True)
train_dataset = dataset['train']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompleteD2PBKW/amazon_us_reviews-train.tfrecord


  0%|          | 0/104975 [00:00<?, ? examples/s]

Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


In [11]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [12]:
print(train_dataset)

<PrefetchDataset shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


In [13]:
len(list(train_dataset))

104975

In [14]:
BUFFER_SIZE = 40000
BATCH_SIZE = 196

In [15]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [16]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'29202487'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Mobile_Electronics'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B00AB6G1AG'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'87543341'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Usa-apple Iphone 5 Iphone Lightning Adapter 8 Pin to 30pin for Iphone 5 Ipod 5 Nano 7th Gen USA'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b"Would not support our iPhone, will charge but will not play music. We were disappointed but glad we didn't pay a lot for it.">, 'review_date': <tf.Tensor: shape=(), dtype=string, numpy=b'2013-12-27'>, 'review_headline': <tf.Tensor: shape=(), dtype=string, numpy=b'Disappointed'>, 'review_id': <tf.Tensor: shape=(), dtype=string, numpy=b'R3KO

In [17]:
for reviews in train_dataset.take(10):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())
  print(review_text.get('star_rating'))
  print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

b"Would not support our iPhone, will charge but will not play music. We were disappointed but glad we didn't pay a lot for it."
tf.Tensor(1, shape=(), dtype=int32)
0
b'fit as advertised and my daughter loves it, no issues and has stuck so far after a month. would by again'
tf.Tensor(4, shape=(), dtype=int32)
1
b'Most of them at least. I ordered 27, and out of that, only one has failed me so far. I recommend for the price alone...'
tf.Tensor(4, shape=(), dtype=int32)
1
b'It states it works for an Iphone, but this is untrue. My does not fit in this case. It is made for an HTC'
tf.Tensor(3, shape=(), dtype=int32)
0
b"I have used five of these now in boat navigation laptops for myself and friends. Very fast lockup, it always has a position before the nav software loads. Very sensitive, while others are having to stick their GPS out side this one is receiving in the cabin. I like the fact it is powered by the USB port and doesn't need a separate battery."
tf.Tensor(5, shape=(), dtype=int32)

In [19]:
# tokenizer = tfds.features.text.Tokenizer()
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
for _, reviews in train_dataset.enumerate():
  #print(reviews)
  review_text=reviews['data']
  reviews_tokens = tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary_set.update(reviews_tokens)

vocab_size = len(vocabulary_set)
vocab_size

73738